In [ ]:
import kagglehub

# Herunterladen des Datensatzes
path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/breast-cancer-wisconsin-data


In [ ]:
import pandas as pd
import os

df = pd.read_csv(path+"/data.csv")
print("\nDataFrame loaded successfully:")

print(df.head())


DataFrame loaded successfully:
         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  texture_worst  perimet

In [ ]:
class Perceptron:
  def __init__(self, weights, bias):
    """ Konstruiert ein neues Perzeptron.

    Args:
      weights: Die initialen Gewichte des Perzeptrons.
      bias: Der initiale Bias des Perzeptrons.
    """
    self.weights = weights
    self.bias = bias

  def heaviside_step_fn(x):
    """Setzt die mathematische Heaviside-Schrittfunktion um.

    Args:
      x: Der Parameter der Funktion.

    Returns:
      1 wenn x>=0 sonst 0
    """
    return 1 if x >= 0 else 0

  def predict(self, input):
    """ Errechnet den Ausgang des Perzeptrons, basierend auf den Eingangswerten,
        Gewichten und dem Bias.

    Args:
      input: Die Eingangswerte des Perzeptrons.

    Returns:
      Der Ausgabewert (0 oder 1) des Perzeptrons.
    """
    sum = self.linear(input)
    return Perceptron.heaviside_step_fn(sum)

  def linear(self, input):
    """ Errechnet den gewichteten Durchschnitt der Eingangswerte plus den Bias.

    Args:
      input: Die Eingangswerte des Perzeptrons.

    Returns:
      Der gewichtete Durchschnitt aller Werte plus der Bias.
    """
    sum = self.bias
    for i in range(len(input)):
      sum += input[i] * self.weights[i]
    return sum

  def train(self, x, y_true, learning_rate = 0.005):
    """ Trainiert das Perzeptron mit den gegebenen Daten.

    Args:
      x: Die Eingabewerte des Perzeptrons.
      y_true: Der gewünschte Ausgabewert des Perzeptrons.
      learning_rate: Die Lernrate des Perzeptrons.

    Returns:
      Den absoluten Fehler des Perzeptrons.
    """
    # Die Vorhersage vor der Bias-/Gewichtsanpassung.
    y_hat = self.predict(x)
    # Der Fehler durch die Vorhersage.
    error = y_true - y_hat

    # Anpassen der Gewichte und des Bias basierend auf dem Fehler,
    # Lernrate und Eingabewert
    self.weights += learning_rate * error * x
    self.bias += learning_rate * error
    return abs(error)

  def __str__(self):
    return f"Gewichte: {self.weights}, Bias: {self.bias}"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

n_features = X_train_np.shape[1]
initial_weights = np.random.rand(n_features)
initial_bias = np.random.rand()

perceptron = Perceptron(initial_weights, initial_bias)

# Definieren der Hyperparameter für das Training
EPOCHS = 100
LEARNING_RATE = 0.01

print("\nStarting Perceptron Training...")

for epoch in range(EPOCHS):
    total_error = 0
    for x,y in zip(X_train_np, y_train_np):
      total_error += perceptron.train(x, y, LEARNING_RATE)

    print(f"Epoch {epoch + 1}, Total Error: {total_error}")

print("Perceptron Training Finished.")


Starting Perceptron Training...
Epoch 1, Total Error: 177
Epoch 2, Total Error: 141
Epoch 3, Total Error: 109
Epoch 4, Total Error: 109
Epoch 5, Total Error: 96
Epoch 6, Total Error: 92
Epoch 7, Total Error: 73
Epoch 8, Total Error: 94
Epoch 9, Total Error: 76
Epoch 10, Total Error: 74
Epoch 11, Total Error: 71
Epoch 12, Total Error: 67
Epoch 13, Total Error: 69
Epoch 14, Total Error: 68
Epoch 15, Total Error: 77
Epoch 16, Total Error: 77
Epoch 17, Total Error: 59
Epoch 18, Total Error: 62
Epoch 19, Total Error: 65
Epoch 20, Total Error: 73
Epoch 21, Total Error: 73
Epoch 22, Total Error: 69
Epoch 23, Total Error: 62
Epoch 24, Total Error: 61
Epoch 25, Total Error: 57
Epoch 26, Total Error: 65
Epoch 27, Total Error: 63
Epoch 28, Total Error: 59
Epoch 29, Total Error: 57
Epoch 30, Total Error: 65
Epoch 31, Total Error: 67
Epoch 32, Total Error: 64
Epoch 33, Total Error: 62
Epoch 34, Total Error: 62
Epoch 35, Total Error: 62
Epoch 36, Total Error: 60
Epoch 37, Total Error: 57
Epoch 38, 

In [ ]:
print("\nFinal Perceptron:", perceptron)

# Evaluieren der Vorhersagen des Test-Datensatzes.
correct_predictions = 0
for x,y in zip(X_test_np, y_test_np):
    y_pred_i = perceptron.predict(x)
    if y_pred_i == y:
        correct_predictions += 1

accuracy = correct_predictions / X_test_np.shape[0]
print(f"\nAccuracy on Test Set: {accuracy * 100:.2f}%")


Final Perceptron: Gewichte: [-4.48755551e+01  2.47742343e+01 -2.21880782e+02 -3.18260148e+01
  4.69556230e-01  1.52082909e+00  2.85234258e+00  1.81180499e+00
  2.58531864e-01  1.61264105e-01 -6.89385775e-02  2.51749671e+00
  9.94237090e+00  9.77173208e+01  8.32829830e-02  1.32205970e+00
  1.01906494e+00  5.86459016e-01  5.69110198e-01  9.36861550e-01
 -4.58613002e+01  6.05224546e+01 -1.61884849e+02  7.43117707e+01
  2.29527394e-01  6.45916468e+00  7.63860848e+00  2.62271313e+00
  9.45685690e-01  7.91180548e-01], Bias: -6.087944009104582

Accuracy on Test Set: 85.96%
